# How to propagate callbacks to child components

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Callbacks](/docs/concepts/#callbacks)
- [Custom callback handlers](/docs/how_to/custom_callbacks)

:::

If you're creating a custom chain that's composed entirely using LCEL, then callbacks will be propagated automatically for you.

If you're using `RunnableLambda`, `RunnableGenerator` or `@tool` to create custom components, `langchain` will attempt to propagate
callbacks on your behalf from those components to any child `Runnables` if possible.

:::{.callout-important}
If you're working in an `async` code base and are using `python<=3.10`, you will need to propagate `config` or `callbacks`  since LangChain is unable to do this automatically. `sync` code has no such limitation.
::

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain

## Sync (All Python Versions)

You should see that the custom call back is invoked twice!

In [75]:
import asyncio
from typing import Any, Dict, List

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.runnables import RunnableLambda

class MyCustomHandler(BaseCallbackHandler):
    """Sync callback handler that can be used to handle callbacks from langchain."""
    def on_chain_start(self, 
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print('---')
        print(f'Invoking custom callback. Recevied inputs: {inputs}')

@RunnableLambda
def foo(inputs):
    return 'world'

@RunnableLambda
def bar(inputs):
    return foo.invoke('goodbye')

bar.invoke('hello', {'callbacks': [MyCustomHandler()]});

---
Invoking custom callback. Recevied inputs: hello
---
Invoking custom callback. Recevied inputs: goodbye


## Async Python >=3.11

The custom callback will be invoked twice!

This code will work correctly even thought `bar` does not propagate config to `foo` does not propagate config!

In [76]:
import sys
sys.version

'3.11.4 (main, Sep 25 2023, 10:06:23) [GCC 11.4.0]'

In [77]:
import asyncio
from typing import Any, Dict, List

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.runnables import RunnableLambda

class AsyncMyCustomHandler(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain."""
    async def on_chain_start(self, 
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print('---')
        print(f'Invoking custom callback. Recevied inputs: {inputs}')

@RunnableLambda
async def foo(inputs):
    return 'world'

@RunnableLambda
async def bar(inputs):
    return await foo.ainvoke('goodbye')

await bar.ainvoke('hello', {'callbacks': [AsyncMyCustomHandler()]})

---
Invoking custom callback. Recevied inputs: hello
---
Invoking custom callback. Recevied inputs: goodbye


'world'

## Async Python <=3.10

### Incorrect

This code will not work!

In [7]:
import sys
sys.version

'3.9.6 (default, Sep 26 2023, 21:46:56) \n[GCC 11.4.0]'

In [8]:
import asyncio
from typing import Any, Dict, List

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.runnables import RunnableLambda

class AsyncMyCustomHandler(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain."""
    async def on_chain_start(self, 
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print('---')
        print(f'Invoking custom callback. Recevied inputs: {inputs}')

@RunnableLambda
async def foo(inputs):
    return 'world'

@RunnableLambda
async def bar(inputs):
    return await foo.ainvoke('goodbye')

await bar.ainvoke('hello', {'callbacks': [AsyncMyCustomHandler()]})

---
Invoking custom callback. Recevied inputs: hello


'world'

### Correct

This code will work correctly across all supported versions of python.

:::{.callout-tip}

This code exposes `config` and propagates it to child component as `foo.ainvoke('goodbye', config=config)`
:::

In [9]:
@RunnableLambda
async def foo(inputs, config):
    return 'world'

@RunnableLambda
async def bar(inputs, config):
    return await foo.ainvoke('goodbye', config=config)

await bar.ainvoke('hello', {'callbacks': [AsyncMyCustomHandler()]})

---
Invoking custom callback. Recevied inputs: hello
---
Invoking custom callback. Recevied inputs: goodbye


'world'